In [82]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import folium
import time
from IPython.display import HTML, display

In [55]:
data = pd.read_csv('AB_NYC_2019.csv')
data.drop(columns=['id', 'name', 'host_id', 'host_name', 'last_review'], axis=1, inplace=True)
print(data['neighbourhood_group'].unique())
print(data['room_type'].unique())

['Brooklyn' 'Manhattan' 'Queens' 'Staten Island' 'Bronx']
['Private room' 'Entire home/apt' 'Shared room']


In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood_group             48895 non-null  object 
 1   neighbourhood                   48895 non-null  object 
 2   latitude                        48895 non-null  float64
 3   longitude                       48895 non-null  float64
 4   room_type                       48895 non-null  object 
 5   price                           48895 non-null  int64  
 6   minimum_nights                  48895 non-null  int64  
 7   number_of_reviews               48895 non-null  int64  
 8   reviews_per_month               38843 non-null  float64
 9   calculated_host_listings_count  48895 non-null  int64  
 10  availability_365                48895 non-null  int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 4.1+ MB


In [52]:
data.groupby('neighbourhood_group').mean()['price'].sort_values()

neighbourhood_group
Bronx             87.496792
Queens            99.517649
Staten Island    114.812332
Brooklyn         124.383207
Manhattan        196.875814
Name: price, dtype: float64

In [57]:
regions = data.groupby('neighbourhood_group').mean()['price'].sort_values().index.to_list()
room_types = data['room_type'].unique()

### Перекодируем районы по принципу Ordinary Encode по мере увеличения средней стоимости жилья

In [59]:
encoded_regions = {val:i+1 for i, val in enumerate(regions)}
encoded_room_t =  {val:i+1 for i, val in enumerate(room_types)}

data = data.replace({'neighbourhood_group':encoded_regions})
data = data.replace({'room_type':encoded_room_t})
data

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,4,Kensington,40.64749,-73.97237,1,149,1,9,0.21,6,365
1,5,Midtown,40.75362,-73.98377,2,225,1,45,0.38,2,355
2,5,Harlem,40.80902,-73.94190,1,150,3,0,NaN,1,365
3,4,Clinton Hill,40.68514,-73.95976,2,89,1,270,4.64,1,194
4,5,East Harlem,40.79851,-73.94399,2,80,10,9,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...
48890,4,Bedford-Stuyvesant,40.67853,-73.94995,1,70,2,0,NaN,2,9
48891,4,Bushwick,40.70184,-73.93317,1,40,4,0,NaN,2,36
48892,5,Harlem,40.81475,-73.94867,2,115,10,0,NaN,1,27
48893,5,Hell's Kitchen,40.75751,-73.99112,3,55,1,0,NaN,6,2


In [84]:
map = folium.Map(location=[40.5,-74], zoom_start = 10)
map.save("map.html")